In [ ]:
# Notebook 출력설정
# 주요 라이브러리 임포트

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.grid'] = True
import seaborn as sns

In [ ]:
DIR_DATASET = "C:/Users/0stix/Datasets/"
NAME_PROJECT = "2203-kaggle-tps2203"
TARGET = "congetion"

In [ ]:
# 고속 데이터셋 평가
def eval_df(X, y, model=0):
    

In [ ]:
from IPython.display import clear_output
clear_output(wait=True)

In [5]:
# 원본 데이터셋 로드
dir_dataset = "C:/Users/0stix/Datasets/"
name_project = '2203-kaggle-tps2203'
df_train = pd.read_csv(dir_dataset+name_project+'/train.csv')
df_test = pd.read_csv(dir_dataset+name_project+'/test.csv')
df_sub = pd.read_csv(dir_dataset+name_project+'/sample_submission.csv')

len_train = len(df_train)
df_all = pd.concat([df_train, df_test], axis=0)
target = 'congetion'

In [29]:
import os
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from pandas.api.types import CategoricalDtype

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

# Algorithms
from xgboost import XGBRegressor
# from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Optuna - Bayesian Optimization 
# import optuna
# from optuna.samplers import TPESampler

# Plotly
# import plotly.express as px
# from plotly.subplots import make_subplots
# import plotly.figure_factory as ff
# import plotly.offline as offline
# import plotly.graph_objs as go

warnings.filterwarnings('ignore')

def load_data():
    data_dir = Path("../input/tabular-playground-series-mar-2022")
    df_train = pd.read_csv(data_dir / "train.csv")
    df_test = pd.read_csv(data_dir / "test.csv")
    # Merge the splits so we can process them together
    df = pd.concat([df_train, df_test])
    return df

df_data = load_data()

FileNotFoundError: [Errno 2] No such file or directory: '..\\input\\tabular-playground-series-mar-2022\\train.csv'

In [17]:
df_data = df_all.copy()

def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
 
    return df

df_data = reduce_mem_usage(df_data)

df_data.time = pd.to_datetime(df_data.time)
df_data['year'] = df_data.time.dt.year
df_data['month'] = df_data.time.dt.month
df_data['week'] = df_data.time.dt.isocalendar().week
df_data['hour'] = df_data.time.dt.hour
df_data['minute'] = df_data.time.dt.minute
df_data['day_of_week'] = df_data.time.dt.day_name()
df_data['day_of_year'] = df_data.time.dt.dayofyear
df_data['is_weekend'] = (df_data.time.dt.dayofweek >= 5).astype("int")
df_data = df_data.set_index('time')

Mem. usage decreased to 27.60 Mb (39.3% reduction)


In [33]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

# def score_dataset(X, y, model=XGBRegressor(tree_method='gpu_hist', predictor='gpu_predictor'), model_2 = CatBoostRegressor(task_type = 'GPU', silent=True)):
def score_dataset(X, y, model=XGBRegressor(), model_2 = CatBoostRegressor(silent=True)):
#def score_dataset(X, y, model=XGBRegressor(), model_2 = CatBoostRegressor(silent=True)):
    # Label encoding is good for XGBoost and RandomForest, but one-hot
    # would be better for models like Lasso or Ridge. The `cat.codes`
    # attribute holds the category levels.
    for colname in X.select_dtypes(["object"]).columns:
        X[colname] = LabelEncoder().fit_transform(X[colname])
    X['week'] = X['week'].astype(int)
    X = X.drop('row_id',axis=1)
    # Metric for TPS Mar22 competition is MAE (Mean Absolute Error)
    score_xgb = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_absolute_error", n_jobs=1
    )
    
    score_cat = cross_val_score(
        model_2, X, y, cv=5, scoring="neg_mean_absolute_error", n_jobs=1
    )
    
    score = -0.5 * (score_xgb.mean() + score_cat.mean())
    return score

#df_data = df_data.reset_index().set_index('row_id')
#df_data = df_data.drop(outliers_index,axis=0)
#df_data = df_data.reset_index().set_index('time')

x = df_data[df_data['congestion'].isnull() == False].copy()
y = pd.DataFrame(x.pop('congestion'))

baseline_score = score_dataset(x, y)
print(f"Baseline score: {baseline_score:.5f} MAE")

Baseline score: 6.80326 MAE


In [34]:
from math import sin, cos, pi, exp
sin_vals = {
    'NB': 0.0,
    'NE': sin(1 * pi/4),
    'EB': 1.0,
    'SE': sin(3 * pi/4),
    'SB': 0.0,
    'SW': sin(5 * pi/4),    
    'WB': -1.0,    
    'NW': sin(7 * pi/4),  
}

cos_vals = {
    'NB': 1.0,
    'NE': cos(1 * pi/4),
    'EB': 0.0,
    'SE': cos(3 * pi/4),
    'SB': -1.0,
    'SW': cos(5 * pi/4),    
    'WB': 0.0,    
    'NW': cos(7 * pi/4),  
}

df_data['sin'] = df_data['direction'].map(sin_vals)
df_data['cos'] = df_data['direction'].map(cos_vals)

In [35]:
x = df_data[df_data['congestion'].isnull() == False].copy()
y = pd.DataFrame(x.pop('congestion'))
baseline_score = score_dataset(x, y)
print(f"Baseline score: {baseline_score:.5f} MAE")

Baseline score: 6.80326 MAE


In [37]:
df_data['roadway'] = df_data.x.astype(str) + df_data.y.astype(str) + df_data.direction.astype(str)
# px.box(df_data[df_data.congestion.isnull() == False], x="roadway", y="congestion", color = 'roadway')

In [38]:
df_data['x_cos_hour'] = df_data.x * df_data.cos * df_data.hour
df_data['y_sen_hour'] = df_data.y * df_data.sin * df_data.hour

df_data = df_data.drop(['year','x','y','direction'], axis=1)

x = df_data[df_data['congestion'].isnull() == False].copy()
y = pd.DataFrame(x.pop('congestion'))
baseline_score = score_dataset(x, y)
print(f"Baseline score: {baseline_score:.5f} MAE")

Baseline score: 6.63368 MAE


In [39]:
df_data = df_data.reset_index()
keys = ['roadway', 'day_of_week','hour', 'minute']

df = df_data.groupby(by=keys).mean().reset_index().set_index(keys)
df['mean congestion'] = df['congestion']
df_data = df_data.merge(df['mean congestion'], how='left', left_on=keys, right_on=keys)

df = df_data.groupby(by=keys).median().reset_index().set_index(keys)
df['median congestion'] = df['congestion']
df_data = df_data.merge(df['median congestion'], how='left', left_on=keys, right_on=keys)

df = df_data.groupby(by=keys).min().reset_index().set_index(keys)
df['min congestion'] = df['congestion']
df_data = df_data.merge(df['min congestion'], how='left', left_on=keys, right_on=keys)

df = df_data.groupby(by=keys).max().reset_index().set_index(keys)
df['max congestion'] = df['congestion']
df_data = df_data.merge(df['max congestion'], how='left', left_on=keys, right_on=keys)

df_data = df_data.set_index('time')

x = df_data[df_data['congestion'].isnull() == False].copy()
y = pd.DataFrame(x.pop('congestion'))
baseline_score = score_dataset(x, y)
print(f"Baseline score: {baseline_score:.5f} MAE")

Baseline score: 6.25381 MAE


In [40]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    #discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

y = df_data[df_data['congestion'].isnull() == False]['congestion']
x = df_data[df_data['congestion'].isnull() == False].drop('congestion', axis=1)
mi_scores = make_mi_scores(x, y)
mi_scores = pd.DataFrame(mi_scores).reset_index().rename(columns={'index':'Feature'})

In [41]:
# fig = px.bar(mi_scores, x='MI Scores', y='Feature', color="MI Scores",
#              color_continuous_scale='darkmint')
# fig.update_layout(height = 750, title_text="Mutual Information Scores",
#                   title_font=dict(size=29, family="Lato, sans-serif"), xaxis={'categoryorder':'category ascending'}, margin=dict(t=80))

NameError: name 'px' is not defined

In [42]:
qualitative = [col for col in df_data if df_data[col].dtype == 'object']
for feature in qualitative:
    df_data[feature] = LabelEncoder().fit_transform(df_data[feature])
df_data = reduce_mem_usage(df_data)

df_data = df_data.drop(['month','minute','week','day_of_week','is_weekend','day_of_year','cos','sin'],axis=1)

df_train = df_data[df_data.congestion.isnull() == False]
df_test = df_data[df_data.congestion.isnull() == True]

from sklearn.model_selection import train_test_split
X = df_train.drop(['congestion','row_id'],axis = 1)
y = df_train['congestion']

Mem. usage decreased to 49.52 Mb (46.0% reduction)


In [43]:
from sklearn.metrics import mean_absolute_error as mae
def objective(trial):
    params = {
        "random_state":trial.suggest_categorical("random_state", [2022]),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.0001, 0.3),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        "n_estimators": 1000,
        "max_depth":trial.suggest_int("max_depth", 4, 16),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "l2_leaf_reg":trial.suggest_float("l2_leaf_reg",1e-8,3e-5),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'task_type': trial.suggest_categorical('task_type', ['GPU']),
        'loss_function': trial.suggest_categorical('loss_function', ['MAE']),
        'eval_metric': trial.suggest_categorical('eval_metric', ['MAE'])
    }

    model = CatBoostRegressor(**params)
    X_train_tmp, X_valid_tmp, y_train_tmp, y_valid_tmp = train_test_split(X, y, test_size=0.3, random_state=42)
    model.fit(
        X_train_tmp, y_train_tmp,
        eval_set=[(X_valid_tmp, y_valid_tmp)],
        early_stopping_rounds=35, verbose=0
    )
        
    y_train_pred = model.predict(X_train_tmp)
    y_valid_pred = model.predict(X_valid_tmp)
    train_mae = mae(y_train_tmp, y_train_pred)
    valid_mae = mae(y_valid_tmp, y_valid_pred)
    
    print(f'MAE of Train: {train_mae}')
    print(f'MAE of Validation: {valid_mae}')
    
    return valid_mae

In [71]:
allow_optimize = 0

In [72]:
TRIALS = 100
TIMEOUT = 3600

if allow_optimize:
    sampler = TPESampler(seed=42)

    study = optuna.create_study(
        study_name = 'cat_parameter_opt',
        direction = 'minimize',
        sampler = sampler,
    )
    study.optimize(objective, n_trials=TRIALS)
    print("Best Score:",study.best_value)
    print("Best trial",study.best_trial.params)
    
    best_params = study.best_params
    
    X_train_tmp, X_valid_tmp, y_train_tmp, y_valid_tmp = train_test_split(X, y, test_size=0.3, random_state=42)
    model_tmp = CatBoostRegressor(**best_params, n_estimators=30000, verbose=1000).fit(X_train_tmp, y_train_tmp, eval_set=[(X_valid_tmp, y_valid_tmp)], early_stopping_rounds=35)

NameError: name 'TPESampler' is not defined

In [48]:
if allow_optimize:
    model = CatBoostRegressor(**best_params, n_estimators=model_tmp.get_best_iteration(), verbose=1000).fit(X, y)
else:
    model = CatBoostRegressor(
        verbose=1000,
        early_stopping_rounds=10,
        #iterations=5000,
        random_state = 2022, learning_rate = 0.0824038781081412, bagging_temperature = 0.03568558360430449, max_depth = 16, 
        random_strength = 47, l2_leaf_reg = 7.459775961819184e-06, min_child_samples = 49, max_bin = 320, od_type = 'Iter', 
        task_type = 'GPU', 
        loss_function = 'MAE', eval_metric = 'MAE'
    ).fit(X, y)    

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 13.0293626	total: 116ms	remaining: 1m 55s
999:	learn: 5.9164225	total: 1m 57s	remaining: 0us


In [49]:
plot_feature_importance(model.get_feature_importance(),X.columns,'CatBoost')

NameError: name 'plot_feature_importance' is not defined

In [50]:
x_test = df_test.drop(['congestion','row_id'],axis = 1).copy()
predictions = model.predict(x_test)
submit_cat = pd.DataFrame({'row_id':df_test.row_id, 'congestion':predictions})
submit_cat = submit_cat.reset_index().drop('time',axis=1).set_index('row_id')

In [67]:
from sklearn.model_selection import GridSearchCV
allow_optimize = 1
if allow_optimize:
    param_grid={
#         'max_depth': [4,5,6,7,8,9],
        'max_depth': [5],
        #'n_estimators': [100,200,300,400,500,600,700,800,900,1000],
        'n_estimators': [75],
#         'min_child_weight' : [1,2,3,4,5,6],
        'min_child_weight' : [1],
        
#       'gpu_id' : [0]
        }

    regressor = XGBRegressor(
#         tree_method = 'gpu_hist', 
#         predictor = 'gpu_predictor'
    )
    CV_regressor = GridSearchCV(regressor, param_grid, cv=3, scoring="neg_mean_absolute_error", n_jobs= -1, return_train_score = True, verbose = 1)
    CV_regressor.fit(X, y)
    
    print("The best hyperparameters are : ","\n")
    print(CV_regressor.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
The best hyperparameters are :  

{'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 75}


In [68]:
if allow_optimize: 
    CV_regressor = CV_regressor.best_estimator_
else:
    CV_regressor = XGBRegressor(
#         tree_method = 'gpu_hist',
#         predictor = 'gpu_predictor', 
#         gpu_id = 0, 
        max_depth = 4, n_estimators = 100)
CV_regressor.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=75, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [ ]:
plot_feature_importance(CV_regressor.feature_importances_,X.columns,'XGBOOST')

In [69]:
predictions = CV_regressor.predict(x_test)
submit_xgb = pd.DataFrame({'row_id':df_test.row_id, 'congestion':predictions})
submit_xgb = submit_xgb.reset_index().drop('time',axis=1).set_index('row_id')

In [70]:
submit = pd.DataFrame({'congestion': submit_cat['congestion']+0*submit_xgb['congestion']})
# special = pd.read_csv('../input/tps-mar-22-special-values/special v2.csv', index_col="row_id")
special = df_sub

special = special[['congestion']].rename(columns={'congestion':'special'})
submit = submit.merge(special, left_index=True, right_index=True, how='left')
submit['special'] = submit['special'].fillna(submit['congestion']).round().astype(int)
submit = submit.drop(['congestion'], axis=1).rename(columns={'special':'congestion'})
submit['congestion'] = round(submit['congestion'])
submit.to_csv('./submission.csv')

In [73]:
X.head()

,hour,roadway,x_cos_hour,y_sen_hour,mean congestion,median congestion,min congestion,max congestion
time,,,,,,,,
1991-04-01,0,0,0.0,0.0,39.555557,35.0,30.0,80.0
1991-04-01,0,1,0.0,0.0,30.259260,29.0,13.0,69.0
1991-04-01,0,2,-0.0,0.0,37.962963,24.0,21.0,91.0
1991-04-01,0,3,0.0,0.0,15.740741,17.0,0.0,26.0
1991-04-01,0,4,0.0,0.0,61.962963,63.0,52.0,72.0
